### Replace audio

![](pictures/DYx0E.png)

```
ffmpeg -i video.mp4 -i audio.wav -map 0:v -map 1:a -c:v copy -shortest output.mp4
```

* Опция `-map` позволяет вам вручную выбирать потоки / треки. Смотрите [FFmpeg Wiki: Map](https://trac.ffmpeg.org/wiki/Map) для получения дополнительной информации.
* В этом примере используется `-c:v copy` для потокового копирования (мультипликации) видео. Перекодирование видео не происходит. Качество сохраняется, а процесс происходит быстро.
    * Если ваш входной аудиоформат совместим с выходным форматом, измените `-c:v copy` на `-c copy` для потокового копирования как видео, так и аудио.
    * Если вы хотите перекодировать видео и аудио, то удалите `-c:v copy` / `-c copy`.
* Опция `-shortest` сделает вывод такой же продолжительности, как и самый короткий ввод.


### Add audio

![](pictures/eyCVy.png)

```
ffmpeg -i video.mkv -i audio.mp3 -map 0 -map 1:a -c:v copy -shortest output.mkv
```

### Mixing/combining two audio inputs into one

![](pictures/2P6Ls.png)


Используйте видео из video.mkv. Микшируйте аудио из video.mkv и audio.m4a с помощью фильтра [amerge filter](http://ffmpeg.org/ffmpeg-filters.html#amerge):

```
ffmpeg -i video.mkv -i audio.m4a -filter_complex "[0:a][1:a]amerge=inputs=2[a]" -map 0:v -map "[a]" -c:v copy -ac 2 -shortest output.mkv
```

Больше информации [FFmpeg Wiki: Audio Channels](https://trac.ffmpeg.org/wiki/AudioChannelManipulation)

### Generate silent audio

```
ffmpeg -i video.mp4 -f lavfi -i anullsrc=channel_layout=stereo:sample_rate=44100 \
-c:v copy -shortest output.mp4
```

Вы можете использовать [фильтр anullsrc](https://ffmpeg.org/ffmpeg-filters.html#anullsrc) для создания бесшумного аудиопотока. Фильтр позволяет вам выбрать желаемую компоновку канала (моно, стерео, 5.1 и т.д.) и частоту дискретизации.

---
---
---

In [18]:
import os
from pathlib import Path
import subprocess
from pprint import pprint
from textwrap import shorten

In [ ]:
# data_folder = Path('a:/')
# folder_name = os.listdir(data_folder)[-1]
# data_folder = os.path.join(data_folder, folder_name)
# # os.listdir(data_folder)

# paths_list = list()
# files_list = list()
# for p,_,f in os.walk(data_folder):
#     if f:
#         paths_list.append(p)
#         files_list.append(f)

# def path_concat(str_path: str, name_list: list[str]) -> list[str]:
#     output_list = list()
#     for name in name_list:
#         tmp = os.path.join(str_path, name)
#         output_list.append(tmp)
#     return output_list

# videos_list = path_concat(paths_list[0], files_list[0])
# audios_list = path_concat(paths_list[1], files_list[1])

# print('Количество видео файлов: ', len(videos_list))
# print('Количество аудил файлов: ', len(audios_list))

In [44]:
# С помощью ffmpeg заменяем/добавляем аудио дорожку
path = Path(videos_list[0]) 
out_rus = os.path.join(path.parent.absolute(), 'out_rus')
os.makedirs(out_rus , exist_ok=True)

for video, audio in zip(videos_list, audios_list):
    out_file = os.path.join(out_rus, os.path.basename(video))
    
    args = [
        'ffmpeg',
        '-i', video,
        '-i', audio,
        '-map', '0:v',
        '-map', '1:a',
        '-c:v', 'copy',
        '-shortest',
        out_file
    ]
    subprocess.run(args, shell=True, check=True)

---
---

In [11]:
folder = os.path.join('a:/', 'youtube')
files_list = os.listdir(folder)
pprint(files_list)
video = os.path.join(folder, files_list[1])
audio = os.path.join(folder, files_list[0])
name, ext = os.path.splitext(os.path.basename(video))
out_file = os.path.join(folder, f'{name}_out{ext}')

['56f712039fdd4a9c8be5709acf13b289.mp3',
 'LLaMa INSTALL WEIGTHS VRAM REDUCTION.webm',
 'LLaMa INSTALL WEIGTHS VRAM REDUCTION_out.webm',
 'НЕКРОМАНТ И ЕГО ВОЙСКО! Diablo IV.f308.webm.part',
 'НЕКРОМАНТ И ЕГО ВОЙСКО! Diablo IV.webm']


In [26]:
path_to_folder = Path('a:/')
file_list = list(path_to_folder.iterdir())
pprint(file_list)
video = file_list[2]
audio = file_list[1]
print(video.name)
print(audio.name)
out_name = shorten(video.name.replace(video.suffix, ''), 30, placeholder='_out').replace(' ', '_') + video.suffix
out_file = path_to_folder.joinpath(out_name)
# print(out_file)
# print(video.name.replace(video.suffix, ''))

[WindowsPath('a:/$RECYCLE.BIN'),
 WindowsPath('a:/af198c392c504a18a1ed2342ce5603f4.mp3'),
 WindowsPath('a:/Police officer fired for pepper-spraying Black and Latino lieutenant during traffic stop [fj66LBel_WQ].webm')]
Police officer fired for pepper-spraying Black and Latino lieutenant during traffic stop [fj66LBel_WQ].webm
af198c392c504a18a1ed2342ce5603f4.mp3
a:\Police_officer_fired_for_out.webm


In [27]:
# С помощью ffmpeg заменяем/добавляем аудио дорожку

args = [
    'ffmpeg', '-y',
    '-i', video,
    '-i', audio,
    '-map', '0',
    '-map', '1:a',
    '-c:v', 'copy',
    '-shortest',
    out_file
]
subprocess.run(args, shell=True, check=True)

CompletedProcess(args=['ffmpeg', '-y', '-i', WindowsPath('a:/Police officer fired for pepper-spraying Black and Latino lieutenant during traffic stop [fj66LBel_WQ].webm'), '-i', WindowsPath('a:/af198c392c504a18a1ed2342ce5603f4.mp3'), '-map', '0', '-map', '1:a', '-c:v', 'copy', '-shortest', WindowsPath('a:/Police_officer_fired_for_out.webm')], returncode=0)